# Interactive Virtual Assistant

This is a virtual assistant that listens for your voice commands and carries out a number of functions including:
- Sending email or sms messages
- Carrying out web searches
- Opening web pages
- Reporting the weather for thousands of cities around the world
- Playing music or videos (from youtube)
- Answering general questions (eg. "who is the president of Ghana?")
- and a few others...

This is a very basic prototype, and still a work in progress. A lot of improvements are possible.

In [ ]:
import os
import re
import math
import time
import smtplib
import requests
import webbrowser
import urllib.parse
import urllib.request
from gtts import gTTS
from weather import Weather
from mutagen.mp3 import MP3
from datetime import datetime
import speech_recognition as sr
from googlesearch import get_page
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from IPython.display import Audio, display

In [ ]:
# List of contacts that the assistant can send email or sms messages to
# A list of some email-to-sms gateways can be found at https://en.wikipedia.org/wiki/SMS_gateway
contacts = {'<name1>': ['<10-digit-number1>', '@<email-to-sms-gateway1>', '<email-address1>'],
            '<name2>': ['<10-digit-number2>', '@<email-to-sms-gateway2>', '<email-address2>'],
            '<name3>': ['<10-digit-number3>', '@<email-to-sms-gateway3>', '<email-address3>']}

In [ ]:
greetings = ['good morning', 'good afternoon', 'good evening', 'good day', 'how are you', 'hi', 'hello', 'what\'s up']
endings = ['bye', 'goodbye', 'good night', 'goodnight', 'later']
msg_commands = ['text', 'sms to', 'email', 'email to']
questions = ['who', 'what', 'when', 'where', 'how']

#### We will define a few functions to carry out commands

In [ ]:
def speak(phrase):
    "Speaks the phrase that is passed as argument"

    print(phrase)

    text_to_speech = gTTS(text = phrase, lang = 'en')
    text_to_speech.save('audio.mp3')
    
    display(Audio(filename = 'audio.mp3', autoplay = True))

In [ ]:
def listen():
    "listens for commands"

    r = sr.Recognizer()

    with sr.Microphone() as source:
        r.pause_threshold = 1
        r.adjust_for_ambient_noise(source, duration = 1)
        print('Ready...')
        audio = r.listen(source)

    try:
        command = r.recognize_google(audio)
        print('You said: ' + command + '\n')

    #loop back to continue to listen for commands if unrecognizable speech is received
    except sr.UnknownValueError:
        print('Did not recognize command')
        command = listen();

    return command.lower()

In [ ]:
def greet(command = ''):
    
    if any(command.startswith(phrase) for phrase in greetings):
        for phrase in greetings:
            if command.startswith(phrase):
                speak(phrase)
                break
    
    if command:
        if 'hello' in command:
            if len(command.split()) == 1:
                speak('hi')
            else:
                new_command = ' '.join(command.split()[1:])
                assistant(new_command)
                return True
        elif 'hi' in command:
            if len(command.split()) == 1:
                speak('hello')
            else:
                assistant(' '.join(command.split()[1:]))
                return True
        elif 'how are you' in command:
            speak('I\'m doing great, I hope you are too!')
        elif 'what\'s up' in command:
            speak('Just chillin')
    else:
        hr = int(datetime.now().time().strftime('%H:%M').split(':')[0])
        if hr < 12:
            speak('Good morning!')
        elif hr > 16:
            speak('Good evening!')
        else:
            speak('Good afternoon!')

In [ ]:
def send_msg(message, receipient, authenticate = False):
    # Method to send email or SMS
    
    # It would be preferable to create a new email address just for this
    # This project uses a gmail server
    sender = '<your email address>'
    
    server = smtplib.SMTP('smtp.gmail.com:587')
    server.starttls()
    
    if authenticate:
        username = input('Username: ')
        password = input('Password: ')
    else:
        username = '<your email address>'
        password = '<your password>'
    
    print('Logging in...')
    server.login(username,password)
    print('Login successful')
    print('\nSending message...')
    server.sendmail(sender, receipient, '\n'+message)
    server.quit()

In [ ]:
def send_message(command):
    
    contact = ''
    
    if 'text' in command or 'sms to' in command:
        query = 'text (.*)' if 'text' in command else 'sms to (.*)'
        reg_ex = re.search(query, command)
        if reg_ex:
            contact = reg_ex.group(1).lower()
        if contact not in contacts:
            speak('Sorry, I don\'t know who {} is'.format(contact))
        else:
            speak('What should I say?')
            message = listen().capitalize()
            receipient = contacts[contact][0] + contacts[contact][1]
            send_msg(message, receipient)
            speak('Message sent to {}'.format(contact.capitalize()))
            
    elif 'email' in command or 'email to' in command:
        query = 'email to (.*)' if 'email to' in command else 'email (.*)'
        reg_ex = re.search(query, command)
        if reg_ex:
            contact = reg_ex.group(1).lower()
        if contact not in contacts:
            speak('Sorry, I don\'t know who {} is'.format(contact))
        else:
            speak('What should I say?')
            message = listen().capitalize()
            receipient = contacts[contact][2]
            send_msg(message, receipient)
            speak('Email sent to {}'.format(contact.capitalize()))

In [ ]:
def open_webpage(command):
    
    reg_ex = re.search('open (.+)', command)
    
    if reg_ex:
        domain = reg_ex.group(1)
        url = 'https://www.' + domain
        speak('Opening ' + domain)
        webbrowser.open(url)
        print('Done!')
    else:
        speak('I\'m sorry, could you repeat that please?')

In [ ]:
def play_youtube(command):
    
        reg_ex = re.search('play (.*)', command)
        query = reg_ex.group(1)
        query_string = urllib.parse.urlencode({'search_query' : query})
        html_content = urllib.request.urlopen('http://www.youtube.com/results?' + query_string)
        search_results = re.findall(r'href=\"\/watch\?v=(.{11})', html_content.read().decode())
        video = search_results[0]
        webbrowser.open('http://www.youtube.com/watch?v=' + video)
        speak('Playing \"{}\" on YouTube'.format(query))

In [ ]:
def get_weather(command):
    
    weather = Weather()

    if 'forecast' in command:
        reg_ex = re.search('forecast (.*)', command)
        if reg_ex:
            day = 'tomorrow'
            index = 1
            phrase = reg_ex.group(1).split()
            city = phrase[1]
            if len(phrase) > 2:
                day = phrase[2]
                if day.lower() == 'today':
                    index = 0
            location = weather.lookup_by_location(city)
            forecasts = location.forecast()
            speak('Weather forecast for {} {}: {}. Temperatures will range between {}°C and {}°C'
                     .format(city.split()[-1].capitalize(), day, forecasts[index].text(), toCelsius(forecasts[index].low()), toCelsius(forecasts[index].high())))     
    else:
        reg_ex = re.search('weather (.*)', command)
        if reg_ex:
            city = reg_ex.group(1)
            location = weather.lookup_by_location(city)
            condition = location.condition()
            speak('Current weather in {}: {}. Temperature: {}°C, or {}°F'
                     .format(city.split()[-1].capitalize(), condition.text(), toCelsius(condition.temp()), condition.temp()))

In [ ]:
def toCelsius(F):
    
    return int(5/9 * (int(F) - 32))

In [ ]:
def tell_joke():
    
    res = requests.get('https://icanhazdadjoke.com/', headers = {"Accept": "application/json"})
        
    if res.status_code == requests.codes.ok:
        speak(str(res.json()['joke']))
    else:
        speak('Oops! I ran out of jokes.')

In [ ]:
def google_search(command):
    
    if 'search for' in command:
        reg_ex = re.search('search for (.*)', command)
        query = reg_ex.group(1)
        webbrowser.open('https://www.google.com/search?q=' + query.replace(' ', '+'))
        speak('Here are some results')
    else:
        reg_ex = re.search('show me (.*)', command)
        query = reg_ex.group(1)
        
        page_query = 'https://www.google.com/search?q=' + '+'.join(query.split(' '))
        page = get_page(page_query)
        page_soup = soup(page, "html.parser")
        
        # Get the first valid link in the first 5 regex results
        result = ''
        for tag in page_soup.find(id = 'search').findAll('h3', attrs = {'class': 'r'})[:5]:
            reg_ex = re.search('url\?q=(.*)', tag.a['href'])
            if reg_ex:
                result = urllib.parse.unquote(reg_ex.group(1).split('&')[0])
                break
        webbrowser.open(result)
        speak('Here you go...')

In [ ]:
def get_answer(command):
    
    if 'how are you' in command:
        greet(command)
        return
    
    for word in questions:
        if word in command:
            reg_ex = re.search(word + ' (.*)', command)
            query = reg_ex.group(0)
            break
    
    page_query = 'https://duckduckgo.com/?q=' + '+'.join(query.split(' '))
    page = get_page(page_query)
    page_soup = soup(page, "html.parser")
    
    try:
        answer = page_soup.find(id = 'zero_click_abstract').text.strip().split('\n')[0]
        speak(answer)
    except:
        page_query = 'https://www.google.com/search?q=' + '+'.join(query.split(' '))
        page = get_page(page_query)
        page_soup = soup(page, "html.parser")
        try:
            answer = page_soup.find(id = 'search').findAll('div')[0].li.text
            if answer:
                if not (answer == 'Cached'):
                    speak(prefix + ' is ' + answer)
                else:
                    answer = page_soup.find(id = 'rhs_block').findAll('div')[0].span.text.replace('Wikipedia', '')
                    speak(answer)
            else:
                speak('I\'m sorry, I don\'t know the answer to that.')
        except:
            speak('Sorry, I could not get you an answer.')

In [ ]:
def sign_off(command):
    
    if 'bye' in command or 'goodbye' in command:
        speak('Goodbye!')
    if 'good night' in command or 'goodnight' in command:
        speak('Good night, sleep tight!')
    if 'later' in command:
        speak('etchry')

### This is the actual assistant

In [ ]:
def assistant(command):
    "if statements for executing commands"
    
    # ======= TASKS =======
    if any(phrase.lower() in command for phrase in msg_commands):
        send_message(command)
            
    elif '. com' in command or '.com' in command:
        open_webpage(command)
        
    elif 'search for' in command or 'show me' in command:
        google_search(command)

    elif 'play' in command:
        play_youtube(command)
        
    elif 'joke' in command:
        tell_joke()

    # ======= QUERIES -=======
    elif 'time' in command:
        speak('The current time is ' + datetime.now().time().strftime('%H:%M'))
        
    elif 'today\'s date' in command:
        speak('Today\'s date is ' + datetime.now().date().strftime('%Y-%m-%d'))
        
    elif 'weather' in command:
        get_weather(command)
        
    elif any(word in command for word in questions):
        get_answer(command)

    # ======= MISC =======
    elif 'thank you' in command:
        speak('You\'re welcome!')
        if len(command.split()) > 2:
            new_command = ' '.join(command.split()[2:])
            return assistant(new_command)
    
    # ======= GREETINGS =======
    elif any(phrase.lower() in command for phrase in greetings):
        greet(command)

    # ======= ENDINGS =======
    elif any(phrase.lower() in command for phrase in endings):
        sign_off(command)
        return False
    
    else:
        speak('Sorry, I don\'t understand that yet')
        
    # wait for the current speech to end
    time.sleep(math.ceil(MP3('audio.mp3').info.length))
    
    return True

### The Assistant comes alive here

In [ ]:
greet()
# loop to keep running until any of the phrases in 'endings' is spoken
# (endings = ['bye', 'goodbye', 'good night', 'goodnight', 'later'])
keep_talking = True
while keep_talking:
    keep_talking = assistant(listen())

### References
- [Python_Message](https://github.com/CrakeNotSnowman/Python_Message/blob/master/sendMessage.py)
- [Desktop Assistant](https://github.com/jg-fisher/desktopAssistant)
- [Googlesearch Python Module](https://github.com/MarioVilas/googlesearch)
- [Python - Search Youtube for Video](https://www.codeproject.com/Articles/873060/Python-Search-Youtube-for-Video)